<a href="https://colab.research.google.com/github/grazder/TTS_HW/blob/master/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [ ]:
!git clone https://github.com/grazder/TTS_HW.git
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz

Cloning into 'TTS_HW'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 275 (delta 68), reused 264 (delta 60), pack-reused 0
Receiving objects: 100% (275/275), 4.72 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (68/68), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2874  100  2874    0     0  17313      0 --:--:-- --:--:-- --:--:-- 17313
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3283    0  3283    0     0  13622      0 --:--:-- --:--:-- --:--:-- 13566
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [ ]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8764, done.
remote: Counting objects: 100% (1149/1149), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 8764 (delta 1007), reused 879 (delta 867), pack-reused 7615
Receiving objects: 100% (8764/8764), 14.42 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (5988/5988), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Processing /content/apex
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/

In [ ]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

  Cloning git://github.com/NVIDIA/dllogger.git (to revision 26a0f8f1958de2c0c460925ff6102a4d2486d6cc) to /tmp/pip-install-oaozvpxv/dllogger_b11b8fad41424441ac09438605a794ca
  Running command git clone -q git://github.com/NVIDIA/dllogger.git /tmp/pip-install-oaozvpxv/dllogger_b11b8fad41424441ac09438605a794ca
  Running command git rev-parse -q --verify 'sha^26a0f8f1958de2c0c460925ff6102a4d2486d6cc'
  Running command git fetch -q git://github.com/NVIDIA/dllogger.git 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
  Running command git checkout -q 26a0f8f1958de2c0c460925ff6102a4d2486d6cc
     |████████████████████████████████| 183 kB 13.5 MB/s 
     |████████████████████████████████| 195 kB 67.4 MB/s 
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5630 sha256=d8fa0014981f11dc30fd56d364162c9d26c6358ea47f8a494bbb651fdecb92c7
  Stored in directory: /root/.cache/pip/wheels/69/76/a4/ae417dad079ab1cae58eb65601ec01d980c5e02915f4f1792f
  Created wheel for librosa: filename=li

In [ ]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

Extracting nvidia_fastpitch_210824.pt ...
OK
Extracting nvidia_waveglow256pyt_fp16.pt ...
OK


### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
!cd TTS_HW && bash scripts/train.sh



AMP=true, 1x5x2 (global batch size 10)

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
train.py:43: UserWarning: PyProf is unavailable
  warnings.warn('PyProf is unavailable')
DLL 2021-12-14 18:04:20.983022 - PARAMETER | output :  ./output
DLL 2021-12-14 18:04:20.983140 - PARAMETER | dataset_path :  libri_training_data
DLL 2021-12-14 18:04:20.983200 - PARAMETER | log_file :  ./output/nvlog.json
DLL 2021-12-14 18:04:20.983254 - PARAMETER | pyprof :  False
DLL 2021-12-14 18:04:20.983445 - PARAMETER | epochs :  1000
DLL 2021-12-14 18:04:20.983518 - PARAMETER | epochs_per_

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
!cd TTS_HW && bash scripts/inference_example.sh


AMP=false, batch_size=32

DLL 2021-12-14 18:04:35.157450 - PARAMETER | input :  phrases/phrase_1_64.txt
DLL 2021-12-14 18:04:35.157510 - PARAMETER | output :  ./output/audio_phrase_1_64.txt
DLL 2021-12-14 18:04:35.157558 - PARAMETER | log_file :  ./output/audio_phrase_1_64.txt/nvlog_infer.json
DLL 2021-12-14 18:04:35.157602 - PARAMETER | save_mels :  False
DLL 2021-12-14 18:04:35.157649 - PARAMETER | cuda :  True
DLL 2021-12-14 18:04:35.157689 - PARAMETER | cudnn_benchmark :  True
DLL 2021-12-14 18:04:35.157728 - PARAMETER | fastpitch :  output/FastPitch_checkpoint_10.pt
DLL 2021-12-14 18:04:35.157767 - PARAMETER | waveglow :  pretrained_models/waveglow/nvidia_waveglow256pyt_fp16.pt
DLL 2021-12-14 18:04:35.157806 - PARAMETER | sigma_infer :  0.9
DLL 2021-12-14 18:04:35.157845 - PARAMETER | denoising_strength :  0.01
DLL 2021-12-14 18:04:35.157884 - PARAMETER | sampling_rate :  22050
DLL 2021-12-14 18:04:35.157930 - PARAMETER | stft_hop_length :  256
DLL 2021-12-14 18:04:35.157980 - PA

In [ ]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [ ]:
import IPython
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [ ]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import transformers
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from tqdm import tqdm
import torchmetrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi

Tue Dec 14 18:14:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

def val_to_class(val):
    if val < 1:
        return 0
    if 1 < val < 2:
        return 1
    return 2

f1_criterion = torchmetrics.F1(num_classes=3)
acc_criterion = torchmetrics.Accuracy(num_classes=3)

class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        id_dict = self.tokenizer.encode_plus(
            self.sentences[index],
            add_special = False,
            max_length=self.max_len,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
            padding='max_length',
        )

        ids = id_dict['input_ids'].squeeze(0)
        masks = id_dict['attention_mask'].squeeze(0)

        label = [val_to_class(v) for v in self.labels[index]]
        label = [0] + label
        if len(label) > len(ids):
            label = label[:len(ids)]
        else:
            label.extend([0] * (len(ids) - len(label)))

        return {
            'ids': torch.tensor(ids).long(),
            'mask': torch.tensor(masks).long(),
            'tags': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    with open(filename) as f:
        file_lines = f.readlines()

    splited_lines = [x.split() for x in file_lines]
    sentences.append([x[3] for x in splited_lines])
    label_proms.append([val_to_class(float(x[4])) for x in splited_lines])
    label_boundaries.append([val_to_class(float(x[5])) for x in splited_lines])

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=3)
model.to(device)
optimizer = torch.optim.Adam(params = model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        optimizer.zero_grad()
        outputs = model(input_ids=batch['ids'].to(device), 
                        attention_mask=batch['mask'].to(device), 
                        labels=batch['tags'].to(device))

        loss = outputs.loss

        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            outputs = model(input_ids=batch["ids"].to(device), 
                            attention_mask=batch["mask"].to(device))
            mask = batch["mask"].view(-1)
            preds = outputs.logits.cpu().view(-1, 3)[mask == 1].softmax(dim=-1)
            tags = batch["tags"].view(-1)[mask == 1]

            accs.append(acc_criterion(preds, tags))
            f1_scores.append(f1_criterion(preds, tags))

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

EPOCH:0


  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
loss; 0.5514410734176636: 100%|██████████| 14/14 [00:07<00:00,  1.82it/s]


Accuracy: 0.7400000095367432
F1 score: 0.7400000095367432
EPOCH:1


loss; 0.48604816198349: 100%|██████████| 14/14 [00:07<00:00,  1.82it/s]


...

Accuracy: 0.7649999856948853
F1 score: 0.7649999856948853
EPOCH:8


loss; 0.10406237840652466: 100%|██████████| 14/14 [00:07<00:00,  1.83it/s]


Accuracy: 0.800000011920929
F1 score: 0.8000000715255737
EPOCH:9


loss; 0.10445965081453323: 100%|██████████| 14/14 [00:07<00:00,  1.83it/s]

Accuracy: 0.8050000071525574
F1 score: 0.8050000667572021


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!